In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics

from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download("punkt")

from string import punctuation
from stop_words import get_stop_words
# from pymorphy2 import MorphAnalyzer

import re

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
max_words = 1000
max_len = 10
num_classes = 1

epochs = 15
batch_size = 512
print_batch_n = 100

In [3]:
train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')

In [4]:
train_df.label.value_counts()


0    29720
1     2242
Name: label, dtype: int64

In [5]:
X_train, X_valid = train_test_split(train_df, test_size=0.25, random_state=42)
X_train.shape, X_valid.shape

((23971, 3), (7991, 3))

In [6]:
sw = set(get_stop_words("en"))
puncts = set(punctuation)
# sw, puncts

In [8]:
nltk.download('omw-1.4')
morpher = WordNetLemmatizer()
morpher.lemmatize('dogs')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


'dog'

In [9]:
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in puncts)
    txt = txt.lower()
    txt = [morpher.lemmatize(word) for word in txt.split() if word not in sw]
    return " ".join(txt)

In [10]:
from tqdm import tqdm 
tqdm.pandas()

X_train.tweet = X_train.tweet.progress_apply(preprocess_text)
X_valid.tweet = X_valid.tweet.progress_apply(preprocess_text)

100%|██████████| 7991/7991 [00:00<00:00, 19926.66it/s]


In [11]:
train_corpus = " ".join(X_train.tweet)
train_corpus = train_corpus.lower()
tokens = word_tokenize(train_corpus)
tokens[:5]

['weekend', 'world', 'really', 'going', 'bonkers']

In [12]:
from nltk.probability import FreqDist
tokens_filtered = [word for word in tokens if word.isalnum()]


dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]  # вычитание 1 для padding
len(tokens_filtered_top)

999

In [13]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [14]:
def text_to_sequence(text, maxlen):
    
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()] 
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    
    return result[-maxlen:] + padding

In [15]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train.tweet])
x_val = np.asarray([text_to_sequence(text, max_len) for text in X_valid.tweet])

x_train.shape, x_val.shape

((23971, 10), (7991, 10))

In [16]:
class GRUFixedLen(nn.Module) :
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        gru_out, ht = self.gru(x)
       
        if self.use_last:
            last_tensor = gru_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(gru_out[:,:], dim=1)
    
        out = self.linear(last_tensor)
        return torch.sigmoid(out)

In [17]:
class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long() 
        self.target = torch.from_numpy(target).long() 
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [18]:
train_dataset = DataWrapper(x_train, X_train.label.values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DataWrapper(x_val, X_valid.label.values)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [20]:
model = GRUFixedLen(max_words).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [21]:
train_f1_score = torchmetrics.F1Score().to(device)
test_f1_score = torchmetrics.F1Score().to(device)


In [22]:
model = model.to(device)
model.train()
th = 0.5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    running_items, running_right, all_f1_score_train = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        train_f1_score(outputs.squeeze(), labels)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()
        all_f1_score_train += train_f1_score.compute().item()
        
    # выводим статистику о процессе обучения (переводим модель в валидацию)
    model.eval()
    
    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
            f'Step [{i + 1}/{len(train_loader)}]. \n' \
            f'Train: Loss: {loss:.3f}. ' \
            f'Acc: {running_right / running_items:.3f}, ' \
            f'f1_score_train: {all_f1_score_train / (i+1):.3f}, ')

    
    running_loss, running_items, running_right, all_f1_score_train = 0.0, 0.0, 0.0, 0.0
    train_f1_score.reset()
    train_loss_history.append(loss)

        # выводим статистику на тестовых данных
    test_running_right, test_running_total, test_loss, all_f1_score_test = 0.0, 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
        
        test_outputs = model(data[0].to(device))
        test_labels = data[1].to(device)

        # подсчет ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
        
        # подсчет метрики на тесте
        test_f1_score(test_outputs.squeeze(), test_labels)
        
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_right += (test_labels == pred_test_labels).sum()
        
        all_f1_score_test += test_f1_score.compute().item()

    test_loss_history.append(test_loss.item())
    print(f'Test:  Loss: {test_loss:.3f}. Acc: {test_running_right / test_running_total:.3f},', end=' ')
    
    print(f'f1_score_test:  {all_f1_score_test / (j+1):.3f}\n')
    test_f1_score.reset()
    
    model.train()
        
print('Training is finished!')

Epoch [1/15]. Step [47/47]. 
Train: Loss: 0.238. Acc: 0.925, f1_score_train: 0.026, 
Test:  Loss: 0.042. Acc: 0.935, f1_score_test:  0.313

Epoch [2/15]. Step [47/47]. 
Train: Loss: 0.239. Acc: 0.940, f1_score_train: 0.365, 
Test:  Loss: 0.033. Acc: 0.945, f1_score_test:  0.439

Epoch [3/15]. Step [47/47]. 
Train: Loss: 0.158. Acc: 0.948, f1_score_train: 0.501, 
Test:  Loss: 0.826. Acc: 0.946, f1_score_test:  0.545

Epoch [4/15]. Step [47/47]. 
Train: Loss: 0.132. Acc: 0.949, f1_score_train: 0.529, 
Test:  Loss: 0.115. Acc: 0.948, f1_score_test:  0.526

Epoch [5/15]. Step [47/47]. 
Train: Loss: 0.190. Acc: 0.953, f1_score_train: 0.558, 
Test:  Loss: 0.014. Acc: 0.947, f1_score_test:  0.552

Epoch [6/15]. Step [47/47]. 
Train: Loss: 0.128. Acc: 0.954, f1_score_train: 0.593, 
Test:  Loss: 0.035. Acc: 0.945, f1_score_test:  0.524

Epoch [7/15]. Step [47/47]. 
Train: Loss: 0.094. Acc: 0.956, f1_score_train: 0.626, 
Test:  Loss: 0.020. Acc: 0.948, f1_score_test:  0.534

Epoch [8/15]. Step [

Результат на мой взгляд получился неплохой, хоть и из-за недостатка времени, я не упел поиграть с моделью и предобработкой, а параметры подсмотрел в гугле и гите. 